In [61]:
from pathlib import Path
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

FILENAME = 'input.txt'


In [57]:
def find_sum(array):
    array = preprocess_array(array)
    number_edges = detect_number_edges(array)
    detected_numbers = extract_numbers(array, number_edges)
    return sum(detected_numbers)

def preprocess_array(array: np.array):
    # replace '.' with -1 and anything that is not a number with -100
    array = np.where(np.logical_or(np.char.isnumeric(array), array == '.'), array, -100)
    array = np.where(array == '.', -1, array)
    array = array.astype(int)
    # add zeros to the left and right of the array to make it easier to check the edges
    array = np.pad(array, ((1, 1), (1, 1)), 'constant', constant_values=-1)
    return array

def detect_number_edges(array: np.array):
    # Get a list of all the adjacent char pairs by row and reshape it to a 2D array of tuples
    view = sliding_window_view(array, (1, 2))
    edge_detect = view.reshape(view.shape[0], view.shape[1], view.shape[2] * view.shape[3])
    # XOR the two cells to detect edges
    edges = np.logical_xor(edge_detect[:, :, 0] >= 0, edge_detect[:, :, 1] >= 0)
    # List the pairwise coordinates of the edges
    pairs = np.argwhere(edges).reshape(-1, 2, 2)
    # Add 1 to both coordinates of each first pair to compensate for the fact that it points to the cell before the edge
    pairs[:, :, 1] += 1
    return pairs

def extract_numbers(array: np.array, number_edges: np.array):
    # extract the numbers from the pairs
    detected = []
    # TODO: use vectorized operations instead of for loop
    for pair in number_edges:
        num = array[pair[0][0], pair[0][1]:pair[1][1]]
        surrounding = array[pair[0][0] - 1:pair[1][0] + 2, pair[0][1] - 1:pair[1][1] + 1]
        # if any of the cells is -1, the number is adjacent to the symbol
        if -100 in surrounding:
            # convert the array of digits to a number
            num = int(''.join(map(str, num)))
            print(num)
            detected.append(num)
    return detected


In [ ]:
array = preprocess_array(np.array([list(line) for line in test.splitlines() if line]))

In [62]:

# Read input file
content = Path(FILENAME).read_text().splitlines()
# Create a 2D array
content = np.array([list(line) for line in content])
# replace '.' with 0 and anything that is not a number with -1
find_sum(content)


664
343
851
164
617
885
250
470
4
407
570
218
654
776
920
753
566
347
979
935
42
311
102
907
723
622
354
266
288
69
486
849
781
978
724
196
767
892
355
815
390
988
704
826
243
796
729
9
490
721
438
272
54
926
481
523
785
766
493
281
579
669
73
639
479
514
375
525
926
120
580
457
325
829
758
662
937
24
749
323
444
583
223
154
965
119
620
347
470
391
183
75
783
209
312
362
667
379
261
228
907
591
178
227
704
771
667
268
543
581
101
471
545
135
178
225
143
973
322
2
239
985
728
612
527
255
435
304
854
261
749
196
694
779
922
76
82
554
991
19
456
582
597
374
707
609
52
483
739
649
973
511
861
20
148
978
282
401
961
499
347
315
819
841
66
820
836
60
456
434
62
908
770
852
317
691
793
411
963
594
45
729
306
148
35
77
134
584
35
589
482
853
668
238
265
471
741
679
807
185
211
507
178
583
561
521
620
865
247
383
183
179
697
935
50
390
917
904
50
96
960
701
464
53
270
647
342
778
684
279
393
151
352
594
732
11
306
805
832
53
953
228
217
37
349
391
739
286
558
516
647
855
353
43
598
798
719
671


509115

In [58]:
test = """
.....664...998........343...............851............................2............414.....................3....................948.164....
......*..................*617....885...*....................-......250.........536..........470...#..................../4......=.....*......
...407...570..218................-.....654........776.....920.........*753...........566......*..347.....61.-979..786........935...42......."""
find_sum(np.array([list(line) for line in test.splitlines() if line]))

664
343
851
164
617
885
250
470
4
407
654
920
753
347
979
935
42


9285

In [48]:
array = np.array([list(line) for line in test.splitlines() if line])
np.logical_or(np.char.isnumeric(array), array == '.')

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
      